### INTERPRETATE MODELS WITH CAPTUM AND TENSORBOARD

In [1]:
import sys
import os

# Include parent dir
SCRIPT_DIR = os.path.dirname(os.path.abspath('play.ipynb'))
PARENT_DIR = os.path.dirname(SCRIPT_DIR)
sys.path.append(PARENT_DIR)

In [2]:
from models.eeg_conformer import Conformer, ConformerConfig
import torch

In [3]:
dict_config = {
    'mlp_ratio': 2,
    'enc_layers': 2,
    'n_head': 4,
    'n_embd': 40,
    'kernel_chan': 64,
    'kernel_temp': 8,
    'pool': 20,
    'pool_hop': 4,
    'block_size': 50,
    'eeg_channels': 64,
    'dropout': 0.4,
    'classifier': True,
    'bias': True,
}
config = ConformerConfig(**dict_config)

In [4]:
mdl = Conformer(config)

Number of parameters: 0.20M


In [5]:
mdl.classif.input_size

240

In [54]:
mdl_folder_path = os.path.join(PARENT_DIR, 'results', 'population', 'models', 'fulsang_data')
models = os.listdir(mdl_folder_path)
print(models)

['CNN', 'Conformer', 'FCNN', 'VLAAI']


In [55]:
# función next para encontrar valores que encajen con el filtro en una lista
model_name = next((m for m in models if 'Conformer' in m), None) 
model_runs = os.listdir(os.path.join(mdl_folder_path, model_name))
mdl_filename = next((m for m in model_runs if 'Conformer' in m), None)
mdl_load_path = os.path.join(mdl_folder_path, model_name, mdl_filename)
mdl_load_path

'c:\\Users\\garce\\Desktop\\proyecto_2024\\Repos_2024\\deepAAD_project\\results\\population\\models\\fulsang_data\\Conformer\\Conformer_epoch=21_acc=0.1524.ckpt'

In [63]:
mdl.load_state_dict(torch.load(mdl_load_path, map_location='cpu'))

<All keys matched successfully>

In [15]:
for name, param in mdl.named_parameters():
    if param.requires_grad:
        print(f'Gradiente de {name}: {param.grad}')

Gradiente de embed.shallownet.0.weight: tensor([[[[ 1.5401e-02,  1.3899e-02,  2.5185e-02,  1.5216e-02,  7.6992e-03,
            7.3828e-03, -3.3370e-03,  2.5587e-02]]],


        [[[ 1.7119e-02,  1.0913e-02, -5.6285e-03,  6.1728e-03,  9.1692e-03,
            1.5231e-02,  8.2132e-04, -7.1625e-03]]],


        [[[-9.1513e-04,  1.8168e-02,  1.9971e-02,  1.8264e-02,  1.5089e-02,
            7.2198e-03,  2.3507e-02,  8.8756e-03]]],


        [[[-2.2926e-03, -1.2945e-02, -5.7032e-03,  3.2311e-04,  4.1329e-03,
           -6.2234e-03,  1.0189e-02,  7.5548e-03]]],


        [[[-2.7467e-02, -2.8648e-02, -3.4736e-02, -3.3215e-02, -3.5013e-02,
           -3.8507e-02, -2.8017e-02, -2.6774e-02]]],


        [[[ 1.5665e-02,  1.3091e-02,  4.7352e-03,  1.2698e-02,  1.0971e-02,
            3.6797e-03, -1.6515e-03, -2.6157e-03]]],


        [[[-1.7020e-02, -1.1336e-02, -1.8421e-02, -1.9811e-02, -1.7549e-02,
           -1.6438e-02, -2.0159e-02, -1.6261e-02]]],


        [[[ 8.4765e-04,  6.4351e-03, -6.372

In [12]:
# compute a forward pass to visuallize gradients
x = torch.randn((128, 64, 50))
target = torch.randn((128))
preds, loss = mdl(x, target)
preds.shape, loss

(torch.Size([128]), tensor(0.0634, grad_fn=<NegBackward0>))

In [14]:
loss.backward()